In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/Users/emiliano/Documents/Programas/Python/DATATHON/df_2021.csv')

/var/folders/tg/ld6r4jms7lx8spbsl_1rb63m0000gn/T/ipykernel_39165/2233145990.py:1: DtypeWarning: Columns (8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/Users/emiliano/Documents/Programas/Python/DATATHON/df_2021.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823855 entries, 0 to 6823854
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   idListaCobro          int64  
 1   idCredito             int64  
 2   consecutivoCobro      int64  
 3   idBanco               int64  
 4   montoExigible         float64
 5   montoCobrar           float64
 6   montoCobrado          float64
 7   fechaCobroBanco       object 
 8   idRespuestaBanco      object 
 9   fechaEnvioCobro       object 
 10  pagare                float64
 11  capital               float64
 12  fechaAperturaCredito  object 
 13  idBancoStr            object 
 14  idRespuestaBancoStr   object 
dtypes: float64(5), int64(4), object(6)
memory usage: 780.9+ MB


In [4]:
df.isnull().sum()

idListaCobro                  0
idCredito                     0
consecutivoCobro              0
idBanco                       0
montoExigible                 0
montoCobrar                   0
montoCobrado                  0
fechaCobroBanco         6504510
idRespuestaBanco             49
fechaEnvioCobro               0
pagare                        0
capital                       0
fechaAperturaCredito          0
idBancoStr                    0
idRespuestaBancoStr     6712519
dtype: int64

In [ ]:
df['fechaEnvioCobro'] = pd.to_datetime(df['fechaEnvioCobro'], errors='coerce')

df = df.sort_values(by=['idCredito', 'fechaEnvioCobro'])

df['acumulado'] = df.groupby('idCredito')['montoCobrado'].cumsum()

df['porcentaje_pagado'] = df['acumulado'] / df['pagare'].replace(0, pd.NA) * 100

df = df.sort_values(by='fechaEnvioCobro')

In [6]:
df['porcentaje_pagado'].describe()

count    6.823855e+06
mean     1.535857e+01
std      1.671051e+01
min      0.000000e+00
25%      2.083333e+00
50%      9.630924e+00
75%      2.385418e+01
max      1.333333e+02
Name: porcentaje_pagado, dtype: float64

In [7]:
df_max_pago = df.loc[df.groupby('idCredito')['porcentaje_pagado'].idxmax()]


In [8]:
df_max_pago['porcentaje_pagado'].describe()

count    51154.000000
mean         9.647224
std         17.797691
min          0.000000
25%          0.000000
50%          1.970658
75%          8.333333
max        133.333333
Name: porcentaje_pagado, dtype: float64

In [ ]:
import plotly.express as px

fig = px.box(
    df_max_pago,
    y="porcentaje_pagado",
    points="all", 
    title="Distribución del Porcentaje Pagado",
    labels={"porcentaje_pagado": "Porcentaje Pagado (%)"},
    template="plotly_dark", 
    height=500,
    width=800
)

fig.update_traces(
    marker=dict(size=4, color="orange"),
    line=dict(color="cyan")
)

fig.update_layout(
    title_font_size=20,
    yaxis_title_font_size=16
)

fig.show()


In [15]:
df_cumplidos = df_max_pago[df_max_pago["porcentaje_pagado"] >= 50]
df_incumplidos = df_max_pago[df_max_pago["porcentaje_pagado"] <= 50]


In [16]:
df_cumplidos['promedio_exceso_pagare_vs_capital'].describe()

count    3186.000000
mean        1.226477
std         0.337309
min         0.475972
25%         1.035600
50%         1.035609
75%         1.392008
max         1.948819
Name: promedio_exceso_pagare_vs_capital, dtype: float64

In [17]:
df_incumplidos['promedio_exceso_pagare_vs_capital'].describe()

count    48065.000000
mean         1.407525
std          0.400726
min          0.198986
25%          1.035607
50%          1.461588
75%          1.781713
max          2.505680
Name: promedio_exceso_pagare_vs_capital, dtype: float64

In [14]:
df_max_pago

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito,idBancoStr,idRespuestaBancoStr,acumulado,porcentaje_pagado,diferencia_relativa_pagare_vs_capital,promedio_exceso_pagare_vs_capital
0,102697,577278,29089112,72,918.43,918.43,0.00,NaN,04,NaT,44084.64,14950.0,2021-01-02,BANORTE,Cuenta Insuficiencia Fondos,0.00,0.000000,1.948805,1.948805
1,102697,577279,29089113,2,1077.65,1077.65,0.00,NaN,04,NaT,51726.73,19720.0,2021-01-02,BANAMEX,Cuenta Insuficiencia Fondos,0.00,0.000000,1.623059,1.623059
2,85819,577281,24701416,72,1843.00,1843.00,1843.00,25/03/2022,00,NaT,88464.00,30000.0,2021-01-02,BANORTE,Domiciliacion Exitosa,1843.00,2.083333,1.948800,1.948800
3,86602,577284,24854764,14,921.50,921.50,0.00,NaN,04,2022-01-04,88464.00,30000.0,2021-01-02,SANTANDER,Cuenta Insuficiencia Fondos,0.00,0.000000,1.948800,1.948800
4,85820,577285,24702100,12,2732.37,2732.37,0.00,NaN,04,NaT,131153.76,50000.0,2021-01-02,BBVA MEXICO,Cuenta Insuficiencia Fondos,0.00,0.000000,1.623075,1.623075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51149,161922,755683,44168227,12,585.33,585.33,585.33,14/05/2025,0.0,NaT,28095.84,10000.0,2025-05-07,BBVA MEXICO,NaN,585.33,2.083333,1.809584,1.809584
51150,161923,755695,44168231,14,2654.48,2654.48,2654.48,15/05/2025,0.0,NaT,127415.04,45350.0,2025-05-07,SANTANDER,NaN,2654.48,2.083333,1.809593,1.809593
51151,162129,755726,44252380,12,3149.99,3149.99,3149.99,19/05/2025,0.0,NaT,163799.48,58300.0,2025-05-08,BBVA MEXICO,NaN,3149.99,1.923077,1.809597,1.809597
51152,162127,755859,44252316,2,2928.79,2928.79,0.00,NaN,4.0,NaT,152297.08,54206.0,2025-05-09,BANAMEX,NaN,0.00,0.000000,1.809598,1.809598


In [ ]:
df_max_pago["diferencia_relativa_pagare_vs_capital"] = (
    (df_max_pago["pagare"] - df_max_pago["capital"]) / df_max_pago["capital"]
)

porcentaje_por_credito = (
    df_max_pago.groupby("idCredito")["diferencia_relativa_pagare_vs_capital"]
    .mean()
    .reset_index()
    .rename(columns={"diferencia_relativa_pagare_vs_capital": "promedio_exceso_pagare_vs_capital"})
)

df_max_pago = df_max_pago.merge(porcentaje_por_credito, on="idCredito", how="left")

In [ ]:
df_max_pago

In [ ]:
print("Cumplidos:", df_cumplidos.shape)
print("Incumplidos:", df_incumplidos.shape)

if not df_cumplidos.empty and not df_incumplidos.empty:
    df_cumplidos["grupo"] = "Cumplidos"
    df_incumplidos["grupo"] = "Incumplidos"
    df_comparacion = pd.concat([df_cumplidos, df_incumplidos])

    fig = px.box(
        df_comparacion,
        x="grupo",
        y="promedio_exceso_pagare_vs_capital",
        points="all",
        title="Cumplidos vs Incumplidos",
        template="plotly_dark"
    )
    fig.show()
else:
    print("Uno de los grupos está vacío. Ajusta los umbrales.")


Cumplidos: (3186, 19)
Incumplidos: (48065, 19)


/var/folders/tg/ld6r4jms7lx8spbsl_1rb63m0000gn/T/ipykernel_39165/3854807827.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/tg/ld6r4jms7lx8spbsl_1rb63m0000gn/T/ipykernel_39165/3854807827.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

